In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

from datetime import date

# Use a service account.
cred = credentials.Certificate('flightsAccount.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

Destinations structure:

In [ ]:
checkDate = 20230524
origin = 'PSR'

doc_ref = db.collection('TestDestinations').document(origin)
doc_ref.set({
    'CheckDate': checkDate,
    'Destinations': ['BGY','TFS']
})

doc_ref = db.collection('TestDestinations').document(origin).collection('Historical').document(f'{checkDate}')
doc_ref.set({
    'CheckDate': checkDate,
    'Origin': origin,
    'Destinations': ['BGY','TFS']
})

In [ ]:
collections = db.collection('TestDestinations').document('VNO').collections()
for collection in collections:
    for doc in collection.stream():
        print(f'{doc.id} => {doc.to_dict()}')

In [ ]:
checkDate = int(str(date.today()).replace('-',''))
# checkDate = 20230524
origin = 'PSR'
ports = ['PSR','VNO','TFS','BGY']


batch = db.batch()

for i in ports:
    ref = db.collection('TestAirpots').document(i)
    batch.set(ref, {
        'Airport': i,
        'AirportName': f'{i}_Port',
        'AirportCity': f'{i}_City',
        'AirportCountry': f'{i}_Country'
    })

batch.commit()

In [9]:
trip = 'VNO-PSR'

doc_ref = db.collection('TestTrips').document(trip)
doc_ref.set({
    'From': trip[:3],
    'To': trip[4:],
    'Active': True,
    'One Way': False,
    'Outbound Date': 20230904,
    'Return Date': 20230924
})

update_time {
  seconds: 1685170194
  nanos: 989850000
}

In [11]:
trip = 'PSR-VNO'

doc_ref = db.collection('TestTrips').document(trip)
doc_ref.set({
    'From': trip[:3],
    'To': trip[4:],
    'Active': True,
    'One Way': False,
    'Outbound Date': 20230703,
    'Return Date': 20230923
})

update_time {
  seconds: 1685170785
  nanos: 759922000
}

In [2]:
def fireTrip(fireDB, trip: str, isActive: bool, outboundDate: int, returnDate: int=0):
    fireMainDoc = fireDB.collection('TestTrips').document(trip)
    fireActiveDoc = fireDB.collection('TestTrips').document(trip).collection('ActiveTrips').document(trip)
    fireMainDoc.set({
        'From': trip[:3],
        'To': trip[4:],
        'Active': isActive
    })
    if returnDate != 0:
        fireActiveDoc.set({
            'From': trip[:3],
            'To': trip[4:],
            'Active': isActive,
            'OutboundDate': outboundDate,
            'OneWay': False,
            'ReturnDate': returnDate
        })
    else:
        fireActiveDoc.set({
            'From': trip[:3],
            'To': trip[4:],
            'Active': isActive,
            'OutboundDate': outboundDate,
            'OneWay': True
        })

tripT = 'VNO-TFS'
fireTrip(db, tripT, True, 0, 0)

In [8]:
from datetime import date

def fireTrip(fireDB, trip: str, created: int, isActive: bool, outboundDate: int, returnDate: int=0):
    if returnDate != 0:
        fireDoc = fireDB.collection('TestTrips').document(f'{trip}_o{outboundDate}-r{returnDate}')
        fireDoc.set({
            'From': trip[:3],
            'To': trip[4:],
            'Active': isActive,
            'OutboundDate': outboundDate,
            'OneWay': False,
            'ReturnDate': returnDate,
            'TripCreated': created
        })
    else:
        fireDoc = fireDB.collection('TestTrips').document(f'{trip}_o{outboundDate}-r0')
        fireDoc.set({
            'From': trip[:3],
            'To': trip[4:],
            'Active': isActive,
            'OutboundDate': outboundDate,
            'OneWay': True,
            'TripCreated': created
        })

tripT = 'VNO-TFS'
fireTrip(db, tripT, int(str(date.today()).replace('-','')), True, 202330904, 202330914)

In [ ]:
docs = db.collection('Airports').stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

In [21]:
docs = db.collection('Airports').get()

In [22]:
existringAirports = []
for doc in docs:
    existringAirports.append(doc.to_dict())

In [24]:
existringAirports

[{'CheckDate': 20230525,
  'AirportCityName': 'Agadir',
  'AirportCode': 'AGA',
  'AirportCountryCode': 'ma',
  'longitude': -9.41307,
  'AirportTimeZone': 'Africa/Casablanca',
  'Currency': 'MAD',
  'latitude': 30.325,
  'AirportCountryName': 'Morocco',
  'AirportName': 'Agadir',
  'AirportSeoName': 'agadir'},
 {'CheckDate': 20230525,
  'AirportCityName': 'Malaga',
  'AirportCode': 'AGP',
  'AirportCountryCode': 'es',
  'longitude': -4.49911,
  'AirportTimeZone': 'Europe/Madrid',
  'Currency': 'EUR',
  'latitude': 36.6749,
  'AirportCountryName': 'Spain',
  'AirportName': 'Malaga',
  'AirportSeoName': 'malaga'},
 {'CheckDate': 20230525,
  'AirportCityName': 'Alghero',
  'AirportCode': 'AHO',
  'AirportCountryCode': 'it',
  'longitude': 8.29077,
  'AirportTimeZone': 'Europe/Rome',
  'Currency': 'EUR',
  'latitude': 40.6321,
  'AirportCountryName': 'Italy',
  'AirportName': 'Alghero',
  'AirportSeoName': 'alghero'},
 {'CheckDate': 20230528,
  'AirportCityName': 'Athens',
  'AirportCode'

In [28]:
#Update single field in document
city_ref = db.collection('Airports').document('ALL_Airports')

# Set the capital field
city_ref.update({'test3': 'what'})

update_time {
  seconds: 1685356450
  nanos: 252426000
}

In [33]:
#add or remove item from array
city_ref = db.collection('Airports').document('ALL_Airports')

# Atomically add a new region to the 'regions' array field.
# city_ref.update({'airports': firestore.ArrayUnion(['PDL'])})

# // Atomically remove a region from the 'regions' array field.
city_ref.update({'airports': firestore.ArrayRemove(['PDL'])})

update_time {
  seconds: 1685356635
  nanos: 50320000
}
transform_results {
  null_value: NULL_VALUE
}